In [1]:
from dotenv import load_dotenv
import asyncio

load_dotenv()

True

In [ ]:
import os

In [13]:
import requests
import json


def stream_with_reasoning(question, api_token):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "deepseek/deepseek-r1",
        "messages": [{"role": "user", "content": question}],
        "stream": True,
        "include_reasoning": True,
    }

    buffer = ""
    is_reasoning = True  # Track whether we're in reasoning or content phase
    print("Reasoning:", end="\n\n", flush=True)

    with requests.post(url, headers=headers, json=payload, stream=True) as r:
        for chunk in r.iter_content(chunk_size=1024, decode_unicode=True):
            buffer += chunk
            while True:
                try:
                    line_end = buffer.find("\n")
                    if line_end == -1:
                        break

                    line = buffer[:line_end].strip()
                    buffer = buffer[line_end + 1 :]

                    if line.startswith("data: "):
                        data = line[6:]
                        if data == "[DONE]":
                            break

                        try:
                            data_obj = json.loads(data)
                            delta = data_obj["choices"][0].get("delta", {})

                            # Check for phase transition
                            if (
                                is_reasoning
                                and "content" in delta
                                and delta.get("content")
                            ):
                                is_reasoning = False
                                print("\n\nResponse:", end="\n\n", flush=True)

                            # Handle reasoning phase
                            if is_reasoning and "reasoning" in delta:
                                reasoning = delta["reasoning"]
                                if reasoning and reasoning.strip():
                                    print(reasoning, end="", flush=True)

                            # Handle content phase
                            elif not is_reasoning and "content" in delta:
                                content = delta["content"]
                                if content and content.strip():
                                    print(content, end="", flush=True)

                        except json.JSONDecodeError:
                            pass
                except Exception as e:
                    print(f"\nError: {str(e)}")
                    break


question = "How would you build the tallest building ever?"
api_token = os.getenv("OPENROUTER_API_KEY")
stream_with_reasoning(question, api_token)

Reasoning:

Okay, so I want to figure out how to build the tallest building ever. Hmm, where do I start? Well, first, I need to think about what makes a building tall and stable. I remember that skyscrapers have foundations that go deep into the ground. Maybe the foundation has to be really strong and secure to support such a massive structure. But how deep would that foundation need to be? I guess it depends on the soil and the location. Maybe somewhere with bedrock close to the surface would be better, like Manhattan?

Then there's the material. Traditional buildings use steel and concrete, right? But for something taller than the Burj Khalifa, which is currently the tallest, maybe we need stronger materials. Wait, the Burj Khalifa uses a bundled tube design, which helps with wind resistance. So the design is crucial. The shape needs to minimize wind forces. Maybe a tapered design?

Also, elevators. The taller the building, the more elevators you need, but they take up a lot of space